# Chapter 5 Data Frame Gymnastics: Joining and Group
This chapter covers
- Joining two data frames together
- Selecting the right type of join for your use case
- Grouping data and understanding the GroupedData transitional object
- Breaking the GroupedData with an aggregation method
- Filling null values in your data frame


## Start a spark session and import logs and logs identifier table

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

# change the account name to your email account
account='sli'

# define a root path to access the data in the DataAnalysisWithPythonAndPySpark
data_path='/net/clusterhn/home/'+account+'/isa460/data/'

spark = (SparkSession.builder.appName("Analyzing tabluar data")
        .config("spark.port.maxRetries", "100")
        .getOrCreate())

# confiture the log level (defaulty is WWARN)
spark.sparkContext.setLogLevel('ERROR')

# import log file
directory=data_path+'/broadcast_logs/'

logs=spark.read.csv(os.path.join(directory, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
                                 sep="|",
                                 header=True,
                                 inferSchema=True,
                                 timestampFormat="yyyy-MM-dd",)

# add Duration second column
logs=logs.withColumn("Duration_seconds", F.col("Duration").substr(1,2).cast("int").alias("dur_hours")*60*60+ 
            F.col("Duration").substr(4,2).cast("int").alias("dur_minutes")*60+
            F.col("Duration").substr(7,2).cast("int").alias("dur_seconds"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/28 15:14:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/28 15:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/28 15:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/09/28 15:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/09/28 15:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# import log identifier table. We only want primary channel (PrimaryFG is 1)

log_identifier=spark.read.csv(os.path.join(directory, "ReferenceTables/LogIdentifier.csv"),
                                 sep="|",
                                 header=True,
                                 inferSchema=True).filter(F.col('PrimaryFG')==1)
log_identifier.printSchema()

root
 |-- LogIdentifierID: string (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- PrimaryFG: integer (nullable = true)



In [3]:
log_identifier.show(5)

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|           13ST|        3157|        1|
|         2000SM|        3466|        1|
|           70SM|        3883|        1|
|           80SM|        3590|        1|
|           90SM|        3470|        1|
+---------------+------------+---------+
only showing top 5 rows



## Joining tables
- Type of join: inner, left, right, full/outer, left_semi, left_anti, cross

- A left semi-join (how="left_semi") is the same as an inner join, but keeps the columns in the left table.
- A left anti-join (how="left_anti") is the opposite of an inner join. It will keep only the records from the left table that do not match the predicate with any record in the right table

In [5]:
# join logs and channels tables. inner join.

logs_and_channels=logs.join(log_identifier, on="LogServiceID", how="inner")

logs_and_channels.columns

['LogServiceID',
 'BroadcastLogID',
 'LogDate',
 'SequenceNO',
 'AudienceTargetAgeID',
 'AudienceTargetEthnicID',
 'CategoryID',
 'ClosedCaptionID',
 'CountryOfOriginID',
 'DubDramaCreditID',
 'EthnicProgramID',
 'ProductionSourceID',
 'ProgramClassID',
 'FilmClassificationID',
 'ExhibitionID',
 'Duration',
 'EndTime',
 'LogEntryDate',
 'ProductionNO',
 'ProgramTitle',
 'StartTime',
 'Subtitle',
 'NetworkAffiliationID',
 'SpecialAttentionID',
 'BroadcastOriginPointID',
 'CompositionID',
 'Producer1',
 'Producer2',
 'Language1',
 'Language2',
 'Duration_seconds',
 'LogIdentifierID',
 'PrimaryFG']

## Naming conventions in the joning world

In [6]:
# what happens if we join the tables with same column name

logs_and_channels_verbose = logs.join(log_identifier, logs["LogServiceID"] == log_identifier["LogServiceID"])

logs_and_channels_verbose.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [7]:
from pyspark.sql.utils import AnalysisException

try:
    logs_and_channels_verbose.select("LogServiceID")
except AnalysisException as err:
    print(err)   

[AMBIGUOUS_REFERENCE] Reference `LogServiceID` is ambiguous, could be: [`LogServiceID`, `LogServiceID`].


### Note:
PySpark happily joins the two data frames but fails when we try to work with the ambiguous column. This is a common situation when working with data that follows the same convention for column naming. To solve this problem, in this section I show three methods, from the easiest to the most general.

In [8]:
# method 1. 
# use the following join. PySpark kept only the first referred column

logs_and_channels = logs.join(log_identifier, "LogServiceID")
 
logs_and_channels.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [9]:
# method 2. Refer each column by adding table name. Drop one of the columns with the same name.

logs_and_channels_verbose = logs.join(
    log_identifier, logs["LogServiceID"] == log_identifier["LogServiceID"]
)
 
logs_and_channels.drop(log_identifier["LogServiceID"]).select(
    "LogServiceID") 

DataFrame[LogServiceID: int]

In [10]:
# method 3. alias() our tables when performing the join

logs_and_channels_verbose = logs.alias("left").join(
    log_identifier.alias("right"),                     
    logs["LogServiceID"] == log_identifier["LogServiceID"],
)
 
logs_and_channels_verbose.drop(F.col("right.LogServiceID")).select(
    "LogServiceID"
)             

DataFrame[LogServiceID: int]

### join two more tables

we will link two additional tables to continue our data discovery and processing. The CategoryID table contains information about the types of programs, and the ProgramClassID table contains the data that allows us to pinpoint the commercials.

In [11]:
# import log file
directory=data_path+'/broadcast_logs/'

logs=spark.read.csv(os.path.join(directory, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
                                 sep="|",
                                 header=True,
                                 inferSchema=True,
                                 timestampFormat="yyyy-MM-dd",)

# add Duration second column
logs=logs.withColumn("Duration_seconds", F.col("Duration").substr(1,2).cast("int").alias("dur_hours")*60*60+ 
            F.col("Duration").substr(4,2).cast("int").alias("dur_minutes")*60+
            F.col("Duration").substr(7,2).cast("int").alias("dur_seconds"))

# import log identifier
log_identifier=spark.read.csv(os.path.join(directory, "ReferenceTables/LogIdentifier.csv"),
                                 sep="|",
                                 header=True,
                                 inferSchema=True).filter(F.col('PrimaryFG')==1)

# join log and channel
logs_and_channels = logs.join(log_identifier, "LogServiceID")

# import category table and select needed columns
cd_category = spark.read.csv(
    os.path.join(directory, "ReferenceTables/CD_Category.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "CategoryID",
    "CategoryCD",
    F.col("EnglishDescription").alias("Category_Description"), 
)

# import program class and select needed columns
cd_program_class = spark.read.csv(
    os.path.join(directory, "ReferenceTables/CD_ProgramClass.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "ProgramClassID",
    "ProgramClassCD",
    F.col("EnglishDescription").alias("ProgramClass_Description"),
)

# join log and channels tables with cd_category and cd_program_class tables

full_log = logs_and_channels.join(cd_category, "CategoryID", how="left").join(
    cd_program_class, "ProgramClassID", how="left")

## Summarizing the data via groupBy and GroupedData
**what are the channels with the greatest and least proportion of commercials?**

list of commercial codes: "COM", "PRC", "PGI", "PRO", "PSA", "MAG", "LOC", "SPO", "MER", "SOL"

In [12]:
# display average program duration by program class

(full_log.groupBy("ProgramClassCD", "ProgramClass_Description")
 .agg(F.sum("Duration_seconds").alias("duration_total"))
 .orderBy("duration_total",ascending=False)
).show(5, False)

+--------------+--------------------------------------+--------------+
|ProgramClassCD|ProgramClass_Description              |duration_total|
+--------------+--------------------------------------+--------------+
|PGR           |PROGRAM                               |20992510      |
|COM           |COMMERCIAL MESSAGE                    |3519163       |
|PFS           |PROGRAM FIRST SEGMENT                 |1344762       |
|SEG           |SEGMENT OF A PROGRAM                  |1205998       |
|PRC           |PROMOTION OF UPCOMING CANADIAN PROGRAM|880600        |
+--------------+--------------------------------------+--------------+
only showing top 5 rows



In [13]:
# limit to the program with commericals. Why there is no output generated?

log1=full_log.filter(F.col("ProgramClassCD").isin(["COM", "PRC", "PGI", "PRO", "PSA", "MAG", "LOC", "SPO", "MER", "SOL"]))

log1.select("ProgramClassCD", "ProgramClass_Description").distinct().show(100,False)

+--------------+------------------------+
|ProgramClassCD|ProgramClass_Description|
+--------------+------------------------+
+--------------+------------------------+



### What are the channels with the greatest and least proportion of commercials?

In [14]:
answer = (
    full_log.groupby("LogIdentifierID")
    .agg(
        F.sum(                                                              
            F.when(                                                        
                F.trim(F.col("ProgramClassCD")).isin(                       
                    ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
                ),                                                          
                F.col("duration_seconds"),                                  
            ).otherwise(0)                                                  
        ).alias("duration_commercial"),                                     
        F.sum("duration_seconds").alias("duration_total"),
    )
    .withColumn(
        "commercial_ratio", F.col(
            "duration_commercial") / F.col("duration_total")
    )
)

answer.show()

+---------------+-------------------+--------------+--------------------+
|LogIdentifierID|duration_commercial|duration_total|    commercial_ratio|
+---------------+-------------------+--------------+--------------------+
|          BRAVO|              22370|        108920| 0.20538009548292324|
|         BBCKID|               3689|         92104|0.040052549292104576|
|           BOOK|              19305|        105885| 0.18232044198895028|
|           CBKT|              16950|        103410| 0.16391064693936758|
|           CBHT|              17319|        103779| 0.16688347353510827|
|          CBAFT|              13839|        100479| 0.13773027199713372|
|           ATN9|              13312|         99792| 0.13339746673080005|
|           MAKE|              23315|        110196|  0.2115775527242368|
|           13ST|              22567|        108982| 0.20707089244095356|
|         BBCCND|              24173|        110578| 0.21860587096890882|
|         ANIMAL|              22860| 

In [15]:
# channels with the most commercial

answer.orderBy(F.desc("commercial_ratio")).show(10)

+---------------+-------------------+--------------+------------------+
|LogIdentifierID|duration_commercial|duration_total|  commercial_ratio|
+---------------+-------------------+--------------+------------------+
|           CIMT|                775|           775|               1.0|
|          TLNSP|              15480|         15480|               1.0|
|           MSET|               2700|          2700|               1.0|
|         TELENO|              17790|         17790|               1.0|
|          HPITV|                 13|            13|               1.0|
|           TANG|               8125|          8125|               1.0|
|           MMAX|              23333|         23582|0.9894410991434145|
|           MPLU|              20587|         20912|0.9844586840091814|
|          INVST|              20094|         20470|0.9816316560820714|
|          ZT�L�|              21542|         21965|0.9807420896881403|
+---------------+-------------------+--------------+------------

In [16]:
# channel with the least commerical

answer.orderBy("commercial_ratio").show(10)

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|           EURO|                  0|          NULL|            NULL|
|          NINOS|                  0|          NULL|            NULL|
|           PLAY|                  0|         86400|             0.0|
|          SNONE|                  0|          1800|             0.0|
|           CFTF|                  0|          1805|             0.0|
|           CKRT|                  0|         14400|             0.0|
|           CFTV|                  0|           102|             0.0|
|           OTN3|                  0|         86400|             0.0|
|           SKIN|                  0|         86400|             0.0|
|           PENT|                  0|         86400|             0.0|
+---------------+-------------------+--------------+----------------+
only showing top 10 

## Deal with null values
dropna(), fillna()


### dropna()
dropna() is pretty easy to use. This data frame method takes three parameters:

- how, which can take the value any or all. If any is selected, PySpark will drop records where at least one of the fields is null. In the case of all, only the records where all fields are null will be removed. By default, PySpark will take the any mode.

- thresh takes an integer value. If set (its default is None), PySpark will ignore the how parameter and only drop the records with less than thresh non-null values.

- subset will take an optional list of columns that dropna() will use to make its decision.

In [17]:
# drop the records that have commerical_ratio is null

answer_no_null=answer.dropna(subset=["commercial_ratio"])

In [18]:
answer_no_null.orderBy("commercial_ratio").show(10)

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|           PLAY|                  0|         86400|             0.0|
|           CFTV|                  0|           102|             0.0|
|           CFTF|                  0|          1805|             0.0|
|           CKRT|                  0|         14400|             0.0|
|           PENT|                  0|         86400|             0.0|
|           OTN3|                  0|         86400|             0.0|
|           SKIN|                  0|         86400|             0.0|
|          SNONE|                  0|          1800|             0.0|
|          ATN13|                  0|         86400|             0.0|
|         TIMESN|                  0|         86400|             0.0|
+---------------+-------------------+--------------+----------------+
only showing top 10 

### fillna()

This data frame method takes two parameters:

- The value, which is a Python int, float, string, or bool. PySpark will only fill the compatible columns; for instance, if we were to fillna("zero"), our commercial_ratio, being a double, would not be filled.
- The same subset parameter we encountered in dropna(). We can limit the scope of our filling to only the columns we want.

In [19]:
nswer_no_null = answer.fillna(0)
 
answer_no_null.orderBy(
    "commercial_ratio", ascending=False).show(5, False)

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|CIMT           |775                |775           |1.0             |
|TLNSP          |15480              |15480         |1.0             |
|MSET           |2700               |2700          |1.0             |
|TELENO         |17790              |17790         |1.0             |
|HPITV          |13                 |13            |1.0             |
+---------------+-------------------+--------------+----------------+
only showing top 5 rows



In [20]:
# Alternative method 
#Filling our numerical records with zero using the fillna() method and a dict

answer_no_null = answer.fillna(
    {"duration_commercial": 0, "duration_total": 0, "commercial_ratio": 0}
)

## Putting everyting together: develop an end-to-end program

In [21]:
#  commercials.py #############################################################
#
# This program computes the commercial ratio for each channel present in the
# dataset.
#
###############################################################################

import os

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(
    "Getting the Canadian TV channels with the highest/lowest proportion of commercials."
).getOrCreate()

spark.sparkContext.setLogLevel("WARN")

###############################################################################
# Reading all the relevant data sources
###############################################################################

# change the account name to your email account
account='sli'

# define a root path to access the data in the DataAnalysisWithPythonAndPySpark
data_path='/net/clusterhn/home/'+account+'/isa460/data/'

DIRECTORY = data_path+'/broadcast_logs/'

logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
    sep="|",
    header=True,
    inferSchema=True,
    timestampFormat="yyyy-MM-dd"
)

log_identifier = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/LogIdentifier.csv"),
    sep="|",
    header=True,
    inferSchema=True,
)

cd_category = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/CD_Category.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "CategoryID",
    "CategoryCD",
    F.col("EnglishDescription").alias("Category_Description"),
)

cd_program_class = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/CD_ProgramClass.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "ProgramClassID",
    "ProgramClassCD",
    F.col("EnglishDescription").alias("ProgramClass_Description"),
)

###############################################################################
# Data processing
###############################################################################

logs = logs.drop("BroadcastLogID", "SequenceNO")

logs = logs.withColumn(
    "duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ),
)

log_identifier = log_identifier.where(F.col("PrimaryFG") == 1)

logs_and_channels = logs.join(log_identifier, "LogServiceID")

full_log = logs_and_channels.join(cd_category, "CategoryID", how="left").join(
    cd_program_class, "ProgramClassID", how="left"
)

full_log.groupby("LogIdentifierID").agg(
    F.sum(
        F.when(
            F.trim(F.col("ProgramClassCD")).isin(
                ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
            ),
            F.col("duration_seconds"),
        ).otherwise(0)
    ).alias("duration_commercial"),
    F.sum("duration_seconds").alias("duration_total"),
).withColumn(
    "commercial_ratio", F.col("duration_commercial") / F.col("duration_total")
).orderBy(
    "commercial_ratio", ascending=False
).show(
    10, False
)

+---------------+-------------------+--------------+------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio  |
+---------------+-------------------+--------------+------------------+
|CIMT           |775                |775           |1.0               |
|TLNSP          |15480              |15480         |1.0               |
|MSET           |2700               |2700          |1.0               |
|TELENO         |17790              |17790         |1.0               |
|HPITV          |13                 |13            |1.0               |
|TANG           |8125               |8125          |1.0               |
|MMAX           |23333              |23582         |0.9894410991434145|
|MPLU           |20587              |20912         |0.9844586840091814|
|INVST          |20094              |20470         |0.9816316560820714|
|ZT�L�          |21542              |21965         |0.9807420896881403|
+---------------+-------------------+--------------+------------

## In Class Exercise

### Exercise 5.5

Using the data from the data/broadcast_logs/Call_Signs.csv (careful: the delimiter here is the comma, not the pipe!), add the Undertaking_Name to our final table to display a human-readable description of the channel.

In [22]:
call_signs = spark.read.csv(
    os.path.join(DIRECTORY, "Call_Signs.csv"),
    sep=",",
    header=True,
    inferSchema=True,
).select("LogIdentifierID", "Undertaking_Name")

In [31]:
call_signs.show(10, False)

+---------------+-------------------------------------------------------+
|LogIdentifierID|Undertaking_Name                                       |
+---------------+-------------------------------------------------------+
|BRAVO          |Bravo!                                                 |
|CBET           |Canadian Broadcasting Corporation, windsor (CBET-DT)   |
|CFTV3          |Southshore Broadcasting Inc., leamington (CFTV-DT)     |
|CHEX           |591987 B.C. Ltd., peterborough (CHEX-DT)               |
|CICT           |Corus Television Limited Partnership, calgary (CICT-DT)|
|CIMT           |Télé Inter-Rives ltée, rivière-du-loup (CIMT-DT)       |
|CKMI           |Corus Television Limited Partnership, quebec (CKMI-DT) |
|FIGHT          |Fight Network                                          |
|SCSD02         |Super Channel (formerly Allarco Entertainment)         |
|SMITH          |Smithsonian Channel (formerly eqhd)                    |
+---------------+---------------------

In [23]:
full_log1=full_log.join(call_signs,'LogIdentifierID', 'inner')

### Exercise 5.6

The government of Canada is asking for your analysis, but they’d like the PRC to be weighted differently. They’d like each PRC second to be considered 0.75 commercial seconds. Modify the program to account for this change.

In [24]:
full_log.columns

['ProgramClassID',
 'CategoryID',
 'LogServiceID',
 'LogDate',
 'AudienceTargetAgeID',
 'AudienceTargetEthnicID',
 'ClosedCaptionID',
 'CountryOfOriginID',
 'DubDramaCreditID',
 'EthnicProgramID',
 'ProductionSourceID',
 'FilmClassificationID',
 'ExhibitionID',
 'Duration',
 'EndTime',
 'LogEntryDate',
 'ProductionNO',
 'ProgramTitle',
 'StartTime',
 'Subtitle',
 'NetworkAffiliationID',
 'SpecialAttentionID',
 'BroadcastOriginPointID',
 'CompositionID',
 'Producer1',
 'Producer2',
 'Language1',
 'Language2',
 'duration_seconds',
 'LogIdentifierID',
 'PrimaryFG',
 'CategoryCD',
 'Category_Description',
 'ProgramClassCD',
 'ProgramClass_Description']

In [26]:
full_log.groupby("LogIdentifierID").agg(
    F.sum(
        F.when(
            F.trim(F.col("ProgramClassCD")).isin(
                ["COM", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
            ),
            F.col("duration_seconds")).when(F.trim(F.col("ProgramClassCD"))=="PRC", F.col("duration_seconds")*0.75)
        .otherwise(0)
    ).alias("duration_commercial"),
    F.sum("duration_seconds").alias("duration_total"),
).withColumn(
    "commercial_ratio", F.col("duration_commercial") / F.col("duration_total")
).orderBy(
    "commercial_ratio", ascending=False
).show(
    10, False
)

+---------------+-------------------+--------------+------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio  |
+---------------+-------------------+--------------+------------------+
|CIMT           |775.0              |775           |1.0               |
|HPITV          |13.0               |13            |1.0               |
|MSET           |2651.25            |2700          |0.9819444444444444|
|TELENO         |17291.25           |17790         |0.971964586846543 |
|TLNSP          |14872.5            |15480         |0.9607558139534884|
|TV5            |10691.5            |11220         |0.9528966131907308|
|CIVM           |11238.25           |11802         |0.9522326724284019|
|INVST          |19390.5            |20470         |0.9472642892037128|
|ONEBMS         |17425.5            |18522         |0.9408001295756397|
|CANALVIE       |19905.0            |21309         |0.9341123468956779|
+---------------+-------------------+--------------+------------